# Questions of Optimization
## Experiments in optimization in Python

### Naomi Ceder
#### 2020-05-9 2 PM CDT, via https://www.twitch.tv/nceder/

**https://naomiceder.tech, @naomiceder**


## Before we start 

This notebook can (will) be found at https://github.com/nceder/exploring_python

*The Quick Python Book* - http://bit.ly/quick-python

PyCon 2020 Online! - https://www.youtube.com/channel/UCMjMBMGt0WJQLeluw6qNJuA

PSF Board elections - https://www.python.org/nominations/elections/

Nominations - https://www.python.org/nominations/2020-python-software-foundation-board/ 

Nominees will be visible once they have closed (i.e., once they all are in)

How to become a PSF member - (PT-BR) https://carolinedantas.com/tutorial/2020/05/21/psf_ptbr.html

(English) https://carolinedantas.com/tutorial/2020/05/21/psf_en.html

(Español) https://carolinedantas.com/tutorial/2020/05/21/psf_es.html


### Do try this at home... just NOT in production. ;-)

## Optimization

“The real problem is that programmers have spent far too much time worrying about efficiency in the wrong places and at the wrong times; **premature optimization is the root of all evil (or at least most of it) in programming.**” 

-- Donald Knuth

If you look, you can always find things optimize 

### Questions of  optimization (not answers)

* 3 factors - processor time, memory footprint, disk access
* You can improve up to two

### Which factor do we care most about

* time (usually) - faster is better, 
* memory - saving memory costs time and requires using disk, but with large amounts of data, we can fill membory
* disk - the more we go to and from disk, the more memory we save and more time we take.



## Testing optimizations

* consider external factors - use of CPU, RAM, i/o, etc by other processes
* order of data can affect sorting, etc
* measure and test


## Strategies for saving disk access

* read/write files in blocks
* group operations that access disk
* cache
* do as much as possible in RAM


### Measuring disk access

Use top or similar tool

## Strategies for saving memory

* Python 3 over Python 2 - generators and other lazy tricks
* Encouraging garbage collection - reference count, scope, etc
* -OO to removed built doc strings
* see MicroPython for ideas

### Measuring memory

In [ ]:
import sys

def size(obj):
    return sys.getsizeof(obj)

In [37]:
a = 0
b= 1000
a_list = [str(x) for x in range(a, b)]
size(a_list)

9024

In [38]:
a_list[-10:]

['990', '991', '992', '993', '994', '995', '996', '997', '998', '999']

In [41]:
b_list  = [str(x*x) for x in range(a,b)]
size(b_list)

9024

In [40]:
b_list[-10:]

['980100',
 '982081',
 '984064',
 '986049',
 '988036',
 '990025',
 '992016',
 '994009',
 '996004',
 '998001']

In [42]:
c_list = []
for a in range(1000):
    c_list.append([str(x*x) for x in range(a,b+1000)])
    

In [45]:
c_list[-1][-10:]

['3960100',
 '3964081',
 '3968064',
 '3972049',
 '3976036',
 '3980025',
 '3984016',
 '3988009',
 '3992004',
 '3996001']

In [46]:
size(c_list)

9024

In [47]:
# a hack based on a stack overflow answer... no guarantees

import sys

def get_size(obj, seen=None):
    """Recursively finds size of objects"""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle
    # self-referential objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, "__dict__"):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, "__iter__") and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size

In [48]:
get_size(a_list)

60914

In [49]:
get_size(b_list)

63561

In [50]:
get_size(c_list)

96316046

## Stragegies for saving time

* Use memory and avoid disk
   * lru_cache
* pre sort
* Use sets and dicitonaries
* Use comprehensions (even map())
* Use C(ython)
* experiment


## Debug flag

* running interpreter with -O sets debug to true, removes asserts, etc
* -OO also strips out  comments


In [68]:
# fibonacci with recursion

def fib_recursive(number):
    """ this is a recursive fibonacci number generator"""

    assert number > 0
    
    if __debug__:
        print(number)
        
    
    if number in [1, 2]:
        result = 1
    else:
        result =  fib_recursive(number - 1) + fib_recursive(number -  2)
    
    return result

### Lessons

* This is why you NEVER should use asserts as an essential part of the code


### Measuring time

#### timeit module

* commandline  - `python -m timeit "expression"`
* callable interface - `import timeit`
* Timer class

#### timeit commandline

`python -m timeit -n <number> -r <repetitions of timer> -s <setup expression> <expression to time>`

In [ ]:
"-".join(str(n) for n in range(100))

In [51]:
def join_digits_100():
    result = "-".join(str(n) for n in range(100))
    return result

In [52]:
def concat_str(n):
    result = ""
    for x in range(n-1):
        result += str(x) + "-"
    result += str(n-1)
    return result

concat_str(100)

'0-1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21-22-23-24-25-26-27-28-29-30-31-32-33-34-35-36-37-38-39-40-41-42-43-44-45-46-47-48-49-50-51-52-53-54-55-56-57-58-59-60-61-62-63-64-65-66-67-68-69-70-71-72-73-74-75-76-77-78-79-80-81-82-83-84-85-86-87-88-89-90-91-92-93-94-95-96-97-98-99'

In [ ]:
# Note that from commandline 10000 runs and 3 repetitions is used by default

$ python -m timeit '"-".join(str(n) for n in range(100))'
100000 loops, best of 3: 18.2 usec per loop
$ python -m timeit '"-".join([str(n) for n in range(100)])'
100000 loops, best of 3: 16.5 usec per loop
$ python -m timeit '"-".join(map(str, range(100)))'
100000 loops, best of 3: 10.8 usec per loop


In [ ]:
import timeit

# using timeit(), the number of runs needs to be specified or 1000000 is used. Can't repeat
a = timeit.timeit('"-".join(str(n) for n in range(100000))', number=100)
print(a)

b = timeit.timeit('"-".join([str(n) for n in range(100000)])', number=100)
print(b)

c = timeit.timeit('"-".join(map(str, range(100000)))', number=100)
print(c)

d = timeit.timeit('concat_str(100000)', setup="from __main__ import concat_str", 
                number=100)
print(d)

In [ ]:
import timeit
 
# using repeat, the number of runs needs to be specified or 1000000 is used. repeats defaults to 3
a = timeit.repeat('"-".join(str(n) for n in range(100))', number=10000, repeat=3)
print(a)

b = timeit.repeat('"-".join([str(n) for n in range(100)])', number=10000, repeat=3)
print(b)

c = timeit.repeat('"-".join(map(str, range(100)))', number=10000, repeat=3)
print(c)

d = timeit.repeat('concat_str(100)', setup="from __main__ import concat_str", number=10000, repeat=3)
print(d)

In [75]:
%timeit concat_str(100)
%timeit join_digits_100()

45.6 µs ± 1.68 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
38.6 µs ± 1.47 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### What about exceptions?

### Are exceptions slow?

* yes, kinda, but... there's a balance
* if there will be a lot of exceptions, it may be better to LYBL
* if exceptions will be... uh... the exception, then they can make code more readable


In [54]:
def test_while_loop():
    i = 0
    length = 1000
    while i < length:
        x = i * i
        i += 1

In [ ]:
class Count:
    def __init__(self, count):
        self.count = count

    def __getitem__(self, key):
        if 0 < key < self.count:
            return key
        else:
            # IndexError raised to iterator
            raise IndexError


def test_count():
    counter = Count(1000)
    # iterator raises StopIteration to end interation
    for i in counter:
        x = i * i

In [55]:
%timeit test_while_loop()
%timeit test_count()

509 µs ± 106 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
5.78 µs ± 1.14 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### Using the right tool....

In [56]:
from collections import deque

a_list = []
a_deque = deque()

def test_list_queue():
    for x in range(10000):
        a_list.insert(x, 0)
    
    for x in range(10000):
        a_list.pop(0)

def test_deque():
    for x in range(10000):
        a_deque.appendleft(x)
    
    for x in range(10000):
        a_deque.popleft()

In [57]:
%timeit test_list_queue()

%timeit test_deque()

62 ms ± 8.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
13.9 ms ± 4.69 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


## caching

* functools - lrucache


In [58]:
from functools import lru_cache

def fib(number):
    if number < 2:
        return number
    return fib(number-1)+fib(number-2) 

@lru_cache(maxsize=None)
def fib_recursive_cache(number):
    if number in [1, 2]:
        result = 1
    else:
        result =  fib_recursive_cache(number - 1) + fib_recursive_cache(number - 2)
    
    return result
    


In [59]:
%timeit fib(10) 
%timeit fib_recursive_cache(10)

123 µs ± 20.7 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
586 ns ± 189 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Cython


`$ pip install cython`

In [60]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [62]:
def fib(number):
    if number < 2:
        return number
    return fib(number-1)+fib(number-2) 

In [63]:
%%cython
def fib_cython(number):
    if number < 2:
        return number
    return fib_cython(number-1)+fib_cython(number-2)

In [64]:
%%cython
cpdef long fib_cython_typed(long number):
    if number < 2:
        return number
    return fib_cython_typed(number-1)+fib_cython_typed(number-2)

In [65]:
# fibonacci via looping

def fib_loop(number):
    cur_fib,next_fib = 1,1
    for i in range(number - 1):
        cur_fib,  next_fib = next_fib, cur_fib + next_fib
    return cur_fib

In [66]:
%%cython

# fibonacci via looping, cython, typed


cpdef long fib_loop_cython_typed(long number):
    cdef long cur_fib, next_fib
    cur_fib, next_fib = 1,1
    for i in range(number - 1):
        cur_fib,next_fib = next_fib,cur_fib+next_fib
    return cur_fib



In [67]:
%timeit fib(10)

%timeit fib_cython(10)
 
%timeit fib_cython_typed(10)

%timeit fib_loop(10)

%timeit fib_loop_cython_typed(10)

170 µs ± 29.5 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
39.8 µs ± 9.45 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
2.66 µs ± 666 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
7.56 µs ± 1.9 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)
524 ns ± 177 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


File fib.pyx
```
def fib(number):
    if number < 2:
        return number
    return fib(number-1)+fib(number-2)
```
File setup.py
```
from distutils.core import setup
from Cython.Build import cythonize

setup(
    ext_modules = cythonize("fib.pyx")
    )
```
Build with:
`$ python setup.py build_ext --inplace`

## Questions?



## Thanks

### Final Notes

[Feedback and suggestions](https://docs.google.com/forms/d/e/1FAIpQLScO28mEaxsHZKFDsPYoctjCMjndgVw2lUNFKvlrqodNNN4uCw/viewform?usp=pp_url&entry.1081536003=Objects+All+the+Way+Down+-+Apr+24,+2020)

This notebook - https://github.com/nceder/exploring_python

*The Quick Python Book*, 3rd ed - http://bit.ly/quick-python

Me - https://naomiceder.tech, @naomiceder

PyCon 2020 Online! - https://www.youtube.com/channel/UCMjMBMGt0WJQLeluw6qNJuA